<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [3]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

stock_ids = stock_n100

mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AUBANK', 'AXISBANK', 'BAJAJFINSV', 'BAJFINANCE',
       'BANKBARODA', 'BATAINDIA', 'BERGEPAINT', 'BHARTIARTL', 'BRITANNIA',
       'CANBK', 'CIPLA', 'COLPAL', 'DABUR', 'DEEPAKNTR', 'DIVISLAB',
       'DMART', 'DRREDDY', 'EICHERMOT', 'EMAMILTD', 'FEDERALBNK',
       'GODREJCP', 'GODREJIND', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCBANK',
       'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO', 'HINDUNILVR', 'IBREALEST',
       'ICICIBANK', 'ICICIPRULI', 'INDIANB', 'INDIGO', 'INDUSINDBK',
       'INDUSTOWER', 'INFY', 'IRCTC', 'ITC', 'JUBLFOOD', 'KOTAKBANK',
       'LT', 'LTIM', 'M&M', 'MARUTI', 'MCDOWELL-N', 'MOTILALOFS',
       'MUTHOOTFIN', 'NAM-INDIA', 'NATIONALUM', 'NAUKRI', 'PAGEIND',
       'PFIZER', 'PIDILITIND', 'PVR', 'RELIANCE', 'SAIL', 'SBILIFE',
       'SBIN', 'SUNPHARMA', 'TATACONSUM', 'TATAMOTORS', 'TATAPOWER',
       'TATASTEEL', 'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'VOLTAS',
       'WHIRLPOOL', 'WIPRO', 'ZEEL'], dtype=object)

In [4]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    59
1    15
Name: InPortfolio, dtype: int64

In [5]:
# consistent compounders
mypf[mypf['InPortfolio'] == 1]['CCIFlag'].value_counts()

0    11
1     4
Name: CCIFlag, dtype: int64

In [6]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [7]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"


  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [8]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 150
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='2y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_150'] = round((stock_df['Close'] - stock_df['150_SMA'])*100/stock_df['150_SMA'],2)
    stock_df['150_SMA_Lag10'] = stock_df['150_SMA'].shift(10)
    stock_df['Slope%_150'] = round((stock_df['150_SMA'] - stock_df['150_SMA_Lag10'])*100/stock_df['150_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    #stock_df.ta.macd(close='Close', fast=12, slow=26, append=True)
    #stock_df['Trend'] = 'Down'
    #stock_df.loc[stock_df['MACD_12_26_9'] - stock_df['MACDs_12_26_9'] > 10, 'Trend'] = 'Up'
    #stock_df.ta.eom(append=True)    
    stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'EOM_14_100000000':'EOM'}, inplace=True)
    stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0) 
    #stock_df['EOM'] = round(stock_df['EOM'],0) 
    #stock_df['EOM'] = '-ve'
    #stock_df.loc[stock_df['EOM_14_100000000'] > 0, 'EOM'] = '+ve'    
    stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df['Upside%52W'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','150_SMA','150_SMA_Lag10','50_SMA_Lag5'
                   #'MACD_12_26_9','MACDh_12_26_9','MACDs_12_26_9','EOM_14_100000000'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]    
    tmp['Curr_PE'] = current_pe
    tmp['EstmtFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['EstmtFlag'].values[0] 
    tmp['DuraFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['DuraFlag'].values[0]   
    tmp['Stage'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Stage'].values[0]      
    df_prec_dev = df_prec_dev.append(tmp)

In [9]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Upside%52W
Date,,,,,,,,,,,
2023-01-23 00:00:00+05:30,18119.0,^NSEI,3.38,0.76,-0.07,18812.0,15294.0,80.29,19.71,51.0,4.0
2023-01-31 00:00:00+05:30,17662.0,^NSEI,0.35,0.90,-0.30,18812.0,15294.0,67.32,32.68,38.0,7.0
2023-02-07 00:00:00+05:30,17722.0,^NSEI,0.32,0.79,-0.39,18812.0,15294.0,69.00,31.00,43.0,6.0
2023-02-14 00:00:00+05:30,17930.0,^NSEI,1.14,0.71,-0.47,18812.0,15294.0,74.92,25.08,51.0,5.0
2023-02-21 00:00:00+05:30,17827.0,^NSEI,0.21,0.70,-0.35,18812.0,15294.0,71.99,28.01,46.0,6.0


In [10]:
# Upside% Calculations
df_prec_dev['Upside%FC'] = round((df_prec_dev['Forecast']-df_prec_dev['Close'])*100/(df_prec_dev['Close']),0)
df_prec_dev['Upside%Avg'] = round((df_prec_dev['Upside%FC'] + df_prec_dev['Upside%52W'])/2,0)
#df_prec_dev['Upside%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Curr_PE']),0)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%FC', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
#df_prec_dev[df_prec_dev['Trend'] == 'Up'].sort_values(by = 'Upside%', ascending=False)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [11]:
# portfolio stocks
cols = ['Stock', 'Close', 'Dev%_150', 'Slope%_150', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Avg_Cost', 'Med_PE', 'Curr_PE', 'Upside%FC', 'Upside%52W', 'Upside%Avg', 'EstmtFlag','DuraFlag','Stage']
df_prec_dev = df_prec_dev[cols]
cols = ['Stock', 'Close', 'Dev%_150', 'Slope%_150', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Avg_Cost', 'Med_PE', 'Curr_PE', 'EstmtFlag','DuraFlag','Stage']

mypf['Stock'] = mypf['Symbol'] + '.NS'
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df['Profit%FC'] = round((df_prec_dev['Forecast']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
tmp_df['Profit%52W'] = round((df_prec_dev['Max']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
tmp_df['Profit%Avg'] = round((tmp_df['Profit%FC'] + tmp_df['Profit%52W'])/2,0)
print('qualified stocks: '+str(tmp_df.shape[0]))

cols = ['Stock', 'Close', 'Dev%_150', 'Slope%_150', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Avg_Cost', 'Med_PE', 'Curr_PE', 'Profit%FC', 'Profit%52W', 'Profit%Avg', 'EstmtFlag','DuraFlag','Stage']
tmp_df = tmp_df[cols]
tmp_df.sort_values(by = 'Profit%Avg', ascending=False)

qualified stocks: 15


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag,Stage
52,VOLTAS.NS,896.0,1.45,-0.91,0.25,1315.0,745.0,26.46,73.54,72.0,976.0,821.0,43.4,84.70,19.0,60.0,40.0,0.0,1.0,D
10,ICICIPRULI.NS,418.0,-16.64,-1.20,-0.67,595.0,402.0,8.52,91.48,36.0,572.0,443.0,60.9,78.40,29.0,34.0,32.0,1.0,0.0,D
8,MOTILALOFS.NS,658.0,-8.54,-0.97,0.00,931.0,651.0,2.36,97.64,39.0,908.0,733.0,19.7,9.08,24.0,27.0,26.0,-1.0,0.0,D
45,TATAPOWER.NS,208.0,-5.90,-0.45,-0.96,287.0,195.0,14.08,85.92,53.0,232.0,206.0,20.4,21.70,13.0,39.0,26.0,1.0,0.0,O
43,DMART.NS,3526.0,-13.42,-0.52,-1.36,4577.0,3231.0,21.89,78.11,43.0,3983.0,3448.0,138.7,97.40,16.0,33.0,24.0,1.0,1.0,T
24,JUBLFOOD.NS,460.0,-16.88,-1.44,-1.63,639.0,430.0,14.59,85.41,44.0,554.0,484.0,87.9,72.10,14.0,32.0,23.0,1.0,1.0,B
31,RELIANCE.NS,2434.0,-3.26,-0.04,-0.85,2811.0,2229.0,35.24,64.76,54.0,2864.0,2311.0,25.3,25.90,24.0,22.0,23.0,1.0,0.0,O
5,BATAINDIA.NS,1413.0,-18.71,-0.98,-1.59,1971.0,1413.0,-0.04,100.04,23.0,1998.0,1632.0,62.3,56.80,22.0,21.0,22.0,1.0,1.0,D
20,MUTHOOTFIN.NS,986.0,-6.82,-0.19,-1.14,1404.0,956.0,6.65,93.35,35.0,1227.0,1082.0,12.5,10.90,13.0,30.0,22.0,-1.0,0.0,B
38,BAJFINANCE.NS,6379.0,-6.08,0.56,-0.27,7720.0,5265.0,45.39,54.61,57.0,7318.0,6154.0,50.8,35.80,19.0,25.0,22.0,1.0,1.0,T


In [12]:
# portfolio ordered based on slope
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag,Stage
52,VOLTAS.NS,896.0,1.45,-0.91,0.25,1315.0,745.0,26.46,73.54,72.0,976.0,821.0,43.4,84.70,19.0,60.0,40.0,0.0,1.0,D
8,MOTILALOFS.NS,658.0,-8.54,-0.97,0.00,931.0,651.0,2.36,97.64,39.0,908.0,733.0,19.7,9.08,24.0,27.0,26.0,-1.0,0.0,D
51,INFY.NS,1580.0,4.05,0.59,0.00,1866.0,1351.0,44.37,55.63,53.0,1742.0,1628.0,23.0,27.70,7.0,15.0,11.0,1.0,1.0,A
38,BAJFINANCE.NS,6379.0,-6.08,0.56,-0.27,7720.0,5265.0,45.39,54.61,57.0,7318.0,6154.0,50.8,35.80,19.0,25.0,22.0,1.0,1.0,T
10,ICICIPRULI.NS,418.0,-16.64,-1.20,-0.67,595.0,402.0,8.52,91.48,36.0,572.0,443.0,60.9,78.40,29.0,34.0,32.0,1.0,0.0,D
31,RELIANCE.NS,2434.0,-3.26,-0.04,-0.85,2811.0,2229.0,35.24,64.76,54.0,2864.0,2311.0,25.3,25.90,24.0,22.0,23.0,1.0,0.0,O
58,BERGEPAINT.NS,571.0,-6.15,-0.16,-0.88,731.0,540.0,16.02,83.98,54.0,605.0,570.0,72.9,62.00,6.0,28.0,17.0,1.0,1.0,B
45,TATAPOWER.NS,208.0,-5.90,-0.45,-0.96,287.0,195.0,14.08,85.92,53.0,232.0,206.0,20.4,21.70,13.0,39.0,26.0,1.0,0.0,O
20,MUTHOOTFIN.NS,986.0,-6.82,-0.19,-1.14,1404.0,956.0,6.65,93.35,35.0,1227.0,1082.0,12.5,10.90,13.0,30.0,22.0,-1.0,0.0,B
11,HDFCLIFE.NS,501.0,-9.56,-0.54,-1.24,612.0,482.0,14.65,85.35,34.0,678.0,544.0,89.3,71.20,25.0,12.0,18.0,1.0,1.0,A


In [13]:
# wait for max profit

tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)].sort_values(by = 'Profit%Avg', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag,Stage
43,DMART.NS,3526.0,-13.42,-0.52,-1.36,4577.0,3231.0,21.89,78.11,43.0,3983.0,3448.0,138.7,97.4,16.0,33.0,24.0,1.0,1.0,T
24,JUBLFOOD.NS,460.0,-16.88,-1.44,-1.63,639.0,430.0,14.59,85.41,44.0,554.0,484.0,87.9,72.1,14.0,32.0,23.0,1.0,1.0,B
5,BATAINDIA.NS,1413.0,-18.71,-0.98,-1.59,1971.0,1413.0,-0.04,100.04,23.0,1998.0,1632.0,62.3,56.8,22.0,21.0,22.0,1.0,1.0,D
38,BAJFINANCE.NS,6379.0,-6.08,0.56,-0.27,7720.0,5265.0,45.39,54.61,57.0,7318.0,6154.0,50.8,35.8,19.0,25.0,22.0,1.0,1.0,T
11,HDFCLIFE.NS,501.0,-9.56,-0.54,-1.24,612.0,482.0,14.65,85.35,34.0,678.0,544.0,89.3,71.2,25.0,12.0,18.0,1.0,1.0,A
58,BERGEPAINT.NS,571.0,-6.15,-0.16,-0.88,731.0,540.0,16.02,83.98,54.0,605.0,570.0,72.9,62.0,6.0,28.0,17.0,1.0,1.0,B
51,INFY.NS,1580.0,4.05,0.59,0.00,1866.0,1351.0,44.37,55.63,53.0,1742.0,1628.0,23.0,27.7,7.0,15.0,11.0,1.0,1.0,A


In [14]:
# keep a watch
tmp_df[((tmp_df['EstmtFlag'] == 0) & (tmp_df['DuraFlag'] != -1)) | ((tmp_df['DuraFlag'] == 0) & (tmp_df['EstmtFlag'] != -1))].sort_values(by = 'Profit%Avg', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag,Stage
52,VOLTAS.NS,896.0,1.45,-0.91,0.25,1315.0,745.0,26.46,73.54,72.0,976.0,821.0,43.4,84.7,19.0,60.0,40.0,0.0,1.0,D
10,ICICIPRULI.NS,418.0,-16.64,-1.20,-0.67,595.0,402.0,8.52,91.48,36.0,572.0,443.0,60.9,78.4,29.0,34.0,32.0,1.0,0.0,D
45,TATAPOWER.NS,208.0,-5.90,-0.45,-0.96,287.0,195.0,14.08,85.92,53.0,232.0,206.0,20.4,21.7,13.0,39.0,26.0,1.0,0.0,O
31,RELIANCE.NS,2434.0,-3.26,-0.04,-0.85,2811.0,2229.0,35.24,64.76,54.0,2864.0,2311.0,25.3,25.9,24.0,22.0,23.0,1.0,0.0,O
12,NAUKRI.NS,3575.0,-9.94,-0.43,-1.24,4865.0,3363.0,14.09,85.91,44.0,4672.0,4154.0,78.8,63.6,12.0,17.0,14.0,1.0,0.0,B


In [15]:
# sell as soon as you can
tmp_df[(tmp_df['EstmtFlag'] == -1) | (tmp_df['DuraFlag'] == -1)].sort_values(by = 'Profit%Avg', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag,Stage
8,MOTILALOFS.NS,658.0,-8.54,-0.97,0.00,931.0,651.0,2.36,97.64,39.0,908.0,733.0,19.7,9.08,24.0,27.0,26.0,-1.0,0.0,D
20,MUTHOOTFIN.NS,986.0,-6.82,-0.19,-1.14,1404.0,956.0,6.65,93.35,35.0,1227.0,1082.0,12.5,10.90,13.0,30.0,22.0,-1.0,0.0,B
34,DIVISLAB.NS,2919.0,-15.99,-1.58,-1.39,4530.0,2751.0,9.42,90.58,41.0,3393.0,3671.0,45.2,32.30,-8.0,23.0,8.0,-1.0,1.0,B


In [16]:
cols = ['Stock', 'Close', 'Dev%_150', 'Slope%_150', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Med_PE', 'Curr_PE','Upside%FC', 'Upside%52W', 'Upside%Avg','EstmtFlag','DuraFlag','Stage']
df_prec_dev = df_prec_dev[cols]

In [17]:
# prospect consistent compounders (CC)
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Upside%Avg'] > 20]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

qualified stocks: 2


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
24,JUBLFOOD.NS,460.0,-16.88,-1.44,-1.63,639.0,430.0,14.59,85.41,44.0,554.0,87.9,72.1,20.0,39.0,30.0,1.0,1.0,B
41,PAGEIND.NS,38649.0,-16.51,-0.66,-1.87,53631.0,37901.0,4.75,95.25,38.0,43791.0,80.8,1000.0,13.0,39.0,26.0,1.0,1.0,O


In [18]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Upside%Avg'] > 20]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

qualified stocks: 5


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
17,WHIRLPOOL.NS,1298.0,-17.75,-1.46,-1.48,1839.0,1254.0,7.51,92.49,37.0,1629.0,72.2,68.90,26.0,42.0,34.0,1.0,0.0,O
9,SBIN.NS,523.0,-7.19,0.71,-1.38,626.0,434.0,46.59,53.41,32.0,716.0,13.2,9.92,37.0,20.0,28.0,1.0,0.0,O
41,PAGEIND.NS,38649.0,-16.51,-0.66,-1.87,53631.0,37901.0,4.75,95.25,38.0,43791.0,80.8,1000.00,13.0,39.0,26.0,1.0,1.0,O
21,BAJAJFINSV.NS,1394.0,-11.13,0.89,-1.39,1805.0,1093.0,42.32,57.68,50.0,1691.0,39.2,37.00,21.0,29.0,25.0,1.0,0.0,D
15,INDUSINDBK.NS,1102.0,-3.69,1.84,-0.59,1264.0,771.0,67.09,32.91,38.0,1402.0,22.8,12.60,27.0,15.0,21.0,1.0,1.0,O


In [19]:
# heavy upside ordered based on slope
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
15,INDUSINDBK.NS,1102.0,-3.69,1.84,-0.59,1264.0,771.0,67.09,32.91,38.0,1402.0,22.8,12.60,27.0,15.0,21.0,1.0,1.0,O
9,SBIN.NS,523.0,-7.19,0.71,-1.38,626.0,434.0,46.59,53.41,32.0,716.0,13.2,9.92,37.0,20.0,28.0,1.0,0.0,O
21,BAJAJFINSV.NS,1394.0,-11.13,0.89,-1.39,1805.0,1093.0,42.32,57.68,50.0,1691.0,39.2,37.00,21.0,29.0,25.0,1.0,0.0,D
17,WHIRLPOOL.NS,1298.0,-17.75,-1.46,-1.48,1839.0,1254.0,7.51,92.49,37.0,1629.0,72.2,68.90,26.0,42.0,34.0,1.0,0.0,O
41,PAGEIND.NS,38649.0,-16.51,-0.66,-1.87,53631.0,37901.0,4.75,95.25,38.0,43791.0,80.8,1000.00,13.0,39.0,26.0,1.0,1.0,O


In [20]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Discount%'] > 79]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Premium%', ascending=True)

qualified stocks: 2


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
41,PAGEIND.NS,38649.0,-16.51,-0.66,-1.87,53631.0,37901.0,4.75,95.25,38.0,43791.0,80.8,1000.0,13.0,39.0,26.0,1.0,1.0,O
17,WHIRLPOOL.NS,1298.0,-17.75,-1.46,-1.48,1839.0,1254.0,7.51,92.49,37.0,1629.0,72.2,68.9,26.0,42.0,34.0,1.0,0.0,O


In [21]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 7


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
70,ITC.NS,382.0,15.48,1.81,1.45,384.0,201.0,99.04,0.96,73.0,350.0,NaN,26.1,-8.0,1.0,-4.0,1.0,0.0,O
55,M&M.NS,1352.0,5.70,1.25,0.69,1388.0,684.0,94.87,5.13,53.0,1475.0,20.5,19.0,9.0,3.0,6.0,1.0,1.0,O
71,LT.NS,2229.0,11.31,1.90,0.28,2273.0,1453.0,94.69,5.31,61.0,2036.0,NaN,31.2,-9.0,2.0,-4.0,1.0,0.0,O
73,BRITANNIA.NS,4503.0,11.25,1.41,0.25,4637.0,2995.0,91.86,8.14,51.0,3935.0,NaN,59.3,-13.0,3.0,-5.0,1.0,0.0,A
65,INDIANB.NS,285.0,18.40,3.73,0.00,305.0,127.0,88.96,11.04,47.0,284.0,7.4,7.0,-0.0,7.0,4.0,1.0,1.0,O
56,DRREDDY.NS,4486.0,3.49,0.14,0.25,4590.0,3669.0,88.72,11.28,60.0,4829.0,NaN,20.6,8.0,2.0,5.0,1.0,1.0,T
48,HDFCBANK.NS,1646.0,7.05,1.24,0.24,1696.0,1281.0,88.07,11.93,51.0,1813.0,26.0,21.0,10.0,3.0,6.0,1.0,0.0,A


In [22]:
# 52W high ordered based on slope
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
70,ITC.NS,382.0,15.48,1.81,1.45,384.0,201.0,99.04,0.96,73.0,350.0,NaN,26.1,-8.0,1.0,-4.0,1.0,0.0,O
55,M&M.NS,1352.0,5.70,1.25,0.69,1388.0,684.0,94.87,5.13,53.0,1475.0,20.5,19.0,9.0,3.0,6.0,1.0,1.0,O
71,LT.NS,2229.0,11.31,1.90,0.28,2273.0,1453.0,94.69,5.31,61.0,2036.0,NaN,31.2,-9.0,2.0,-4.0,1.0,0.0,O
56,DRREDDY.NS,4486.0,3.49,0.14,0.25,4590.0,3669.0,88.72,11.28,60.0,4829.0,NaN,20.6,8.0,2.0,5.0,1.0,1.0,T
73,BRITANNIA.NS,4503.0,11.25,1.41,0.25,4637.0,2995.0,91.86,8.14,51.0,3935.0,NaN,59.3,-13.0,3.0,-5.0,1.0,0.0,A
48,HDFCBANK.NS,1646.0,7.05,1.24,0.24,1696.0,1281.0,88.07,11.93,51.0,1813.0,26.0,21.0,10.0,3.0,6.0,1.0,0.0,A
65,INDIANB.NS,285.0,18.40,3.73,0.00,305.0,127.0,88.96,11.04,47.0,284.0,7.4,7.0,-0.0,7.0,4.0,1.0,1.0,O


In [23]:
# prospects turning around (exhaustive)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] > -.5) & (tmp_df['Slope%_150'] < 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 3


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
35,HEROMOTOCO.NS,2521.0,-4.88,-0.42,-0.68,2839.0,2089.0,57.54,42.46,41.0,2903.0,NaN,19.4,15.0,13.0,14.0,1.0,0.0,T
23,TATACONSUM.NS,726.0,-6.90,-0.26,-0.93,849.0,655.0,36.70,63.30,42.0,871.0,NaN,64.0,20.0,17.0,18.0,1.0,1.0,O
46,ASIANPAINT.NS,2817.0,-10.43,-0.19,-1.34,3566.0,2576.0,24.35,75.65,48.0,3131.0,73.4,70.7,11.0,27.0,19.0,1.0,1.0,T


In [24]:
# prospects turning around (compact)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] < 0) & (tmp_df['Slope%_50'] > 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%52W', ascending=False)

qualified stocks: 0


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage


In [25]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_150'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 1


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
56,DRREDDY.NS,4486.0,3.49,0.14,0.25,4590.0,3669.0,88.72,11.28,60.0,4829.0,NaN,20.6,8.0,2.0,5.0,1.0,1.0,T


In [26]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_150'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 0


,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage


In [27]:
# star stocks performance

star_stocks = ['BAJFINANCE.NS', 'EICHERMOT.NS', 'FEDERALBNK.NS', 'CANBK.NS',
       'AUBANK.NS', 'TCS.NS', 'INDUSINDBK.NS', 'TECHM.NS', 'ICICIBANK.NS',
       'AXISBANK.NS', 'HCLTECH.NS', 'WIPRO.NS', 'TATAPOWER.NS', 'HDFC.NS',
       'HDFCBANK.NS', 'HINDUNILVR.NS', 'EMAMILTD.NS', 'INDIANB.NS',
       'BANKBARODA.NS', 'INFY.NS', 'KOTAKBANK.NS', 'SBIN.NS',
       'TATACONSUM.NS', 'ITC.NS', 'LT.NS']

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
9,SBIN.NS,523.0,-7.19,0.71,-1.38,626.0,434.0,46.59,53.41,32.0,716.0,13.2,9.92,37.0,20.0,28.0,1.0,0.0,O
45,TATAPOWER.NS,208.0,-5.90,-0.45,-0.96,287.0,195.0,14.08,85.92,53.0,232.0,20.4,21.70,12.0,38.0,25.0,1.0,0.0,O
15,INDUSINDBK.NS,1102.0,-3.69,1.84,-0.59,1264.0,771.0,67.09,32.91,38.0,1402.0,22.8,12.60,27.0,15.0,21.0,1.0,1.0,O
18,AXISBANK.NS,844.0,0.72,1.55,-0.77,962.0,625.0,65.00,35.00,31.0,1060.0,31.3,12.60,26.0,14.0,20.0,1.0,0.0,A
14,ICICIBANK.NS,854.0,-2.82,0.91,-0.80,953.0,650.0,67.39,32.61,46.0,1094.0,24.2,18.70,28.0,12.0,20.0,1.0,1.0,O
26,EICHERMOT.NS,3274.0,-2.67,0.65,-0.09,3850.0,2241.0,64.21,35.79,52.0,3916.0,40.1,34.10,20.0,18.0,19.0,1.0,1.0,T
38,BAJFINANCE.NS,6379.0,-6.08,0.56,-0.27,7720.0,5265.0,45.39,54.61,57.0,7318.0,50.8,35.80,15.0,21.0,18.0,1.0,1.0,T
39,BANKBARODA.NS,160.0,5.20,2.63,-0.57,193.0,90.0,67.86,32.14,39.0,183.0,13.5,7.08,14.0,21.0,18.0,1.0,1.0,A
36,CANBK.NS,279.0,1.07,2.17,-0.65,337.0,175.0,64.17,35.83,36.0,322.0,8.8,5.15,15.0,21.0,18.0,1.0,1.0,T
23,TATACONSUM.NS,726.0,-6.90,-0.26,-0.93,849.0,655.0,36.70,63.30,42.0,871.0,NaN,64.00,20.0,17.0,18.0,1.0,1.0,O


In [28]:
# star stocks ordered based on slope
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
70,ITC.NS,382.0,15.48,1.81,1.45,384.0,201.0,99.04,0.96,73.0,350.0,NaN,26.10,-8.0,1.0,-4.0,1.0,0.0,O
67,TCS.NS,3451.0,7.44,0.90,0.69,3695.0,2915.0,68.72,31.28,51.0,3385.0,28.1,31.00,-2.0,7.0,2.0,1.0,0.0,O
63,HCLTECH.NS,1105.0,9.32,1.29,0.46,1150.0,857.0,84.71,15.29,47.0,1117.0,16.3,20.80,1.0,4.0,2.0,1.0,1.0,A
71,LT.NS,2229.0,11.31,1.90,0.28,2273.0,1453.0,94.69,5.31,61.0,2036.0,NaN,31.20,-9.0,2.0,-4.0,1.0,0.0,O
48,HDFCBANK.NS,1646.0,7.05,1.24,0.24,1696.0,1281.0,88.07,11.93,51.0,1813.0,26.0,21.00,10.0,3.0,6.0,1.0,0.0,A
47,HDFC.NS,2666.0,5.81,1.23,0.04,2775.0,2053.0,84.96,15.04,48.0,2940.0,23.0,19.50,10.0,4.0,7.0,1.0,0.0,A
65,INDIANB.NS,285.0,18.40,3.73,0.00,305.0,127.0,88.96,11.04,47.0,284.0,7.4,7.00,-0.0,7.0,4.0,1.0,1.0,O
25,FEDERALBNK.NS,127.0,1.07,1.59,0.00,140.0,82.0,78.19,21.81,38.0,152.0,11.0,9.62,20.0,10.0,15.0,1.0,0.0,A
51,INFY.NS,1580.0,4.05,0.59,0.00,1866.0,1351.0,44.37,55.63,53.0,1742.0,23.0,27.70,10.0,18.0,14.0,1.0,1.0,A
26,EICHERMOT.NS,3274.0,-2.67,0.65,-0.09,3850.0,2241.0,64.21,35.79,52.0,3916.0,40.1,34.10,20.0,18.0,19.0,1.0,1.0,T


In [29]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] != 1) | (tmp_df['DuraFlag'] == -1)]
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
0,IBREALEST.NS,54.0,-29.55,0.00,-4.11,118.0,54.0,0.39,99.61,24.0,95.0,11.3,1000.00,76.0,119.0,98.0,-1.0,-1.0,B
2,NAM-INDIA.NS,215.0,-19.25,-1.50,-2.05,340.0,215.0,-0.16,100.16,28.0,322.0,29.9,19.10,50.0,58.0,54.0,-1.0,1.0,O
3,ZEEL.NS,211.0,-15.40,0.00,-2.16,299.0,206.0,5.00,95.00,32.0,303.0,24.6,35.80,44.0,42.0,43.0,-1.0,1.0,O
1,GODREJIND.NS,419.0,-5.84,0.00,-1.37,533.0,405.0,10.94,89.06,39.0,638.0,39.5,15.50,52.0,27.0,40.0,0.0,1.0,B
13,INDUSTOWER.NS,164.0,-14.53,-1.56,-1.70,238.0,137.0,26.83,73.17,47.0,213.0,NaN,16.20,30.0,45.0,38.0,-1.0,1.0,O
4,EMAMILTD.NS,390.0,-13.30,-0.67,-1.20,512.0,386.0,3.29,96.71,35.0,550.0,49.6,20.40,41.0,31.0,36.0,-1.0,1.0,T
28,HINDALCO.NS,432.0,-0.35,1.39,-0.86,626.0,314.0,37.66,62.34,38.0,514.0,10.4,8.28,19.0,45.0,32.0,-1.0,1.0,A
49,NATIONALUM.NS,82.0,7.96,1.32,0.00,126.0,66.0,26.75,73.25,58.0,90.0,10.3,7.67,10.0,54.0,32.0,0.0,1.0,O
7,PFIZER.NS,3820.0,-10.11,-0.33,-1.54,4599.0,3815.0,0.70,99.30,33.0,5297.0,36.5,114.00,39.0,20.0,30.0,0.0,1.0,O
16,PVR.NS,1654.0,-8.24,-0.78,-1.23,2193.0,1498.0,22.50,77.50,43.0,2106.0,51.2,1000.00,27.0,33.0,30.0,1.0,-1.0,O


In [30]:
# cross-check ordered based on slope
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

,Stock,Close,Dev%_150,Slope%_150,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag,Stage
68,TECHM.NS,1134.0,9.29,0.67,0.58,1520.0,935.0,34.09,65.91,71.0,1113.0,18.7,21.20,-2.0,34.0,16.0,-1.0,1.0,B
27,TATAMOTORS.NS,436.0,1.75,0.23,0.48,494.0,372.0,52.87,47.13,53.0,517.0,19.2,1000.00,19.0,13.0,16.0,1.0,-1.0,B
62,GODREJCP.NS,937.0,5.78,0.68,0.44,948.0,665.0,96.18,3.82,56.0,958.0,NaN,57.80,2.0,1.0,2.0,-1.0,0.0,A
60,WIPRO.NS,403.0,0.19,0.00,0.25,605.0,374.0,12.45,87.55,50.0,425.0,19.0,19.50,5.0,50.0,28.0,-1.0,0.0,O
64,ULTRACEMCO.NS,7341.0,8.57,1.46,0.17,7427.0,5148.0,96.21,3.79,60.0,7420.0,33.7,35.30,1.0,1.0,1.0,-1.0,1.0,O
53,HAVELLS.NS,1217.0,-1.80,0.08,0.17,1388.0,1065.0,46.97,53.03,56.0,1328.0,63.7,71.50,9.0,14.0,12.0,-1.0,1.0,T
66,SAIL.NS,87.0,6.28,1.22,0.00,108.0,63.0,53.67,46.33,53.0,85.0,6.9,11.00,-2.0,24.0,11.0,-1.0,1.0,O
61,TATASTEEL.NS,113.0,4.77,0.93,0.00,124.0,44.0,86.44,13.56,46.0,118.0,6.0,7.82,4.0,10.0,7.0,-1.0,0.0,O
49,NATIONALUM.NS,82.0,7.96,1.32,0.00,126.0,66.0,26.75,73.25,58.0,90.0,10.3,7.67,10.0,54.0,32.0,0.0,1.0,O
33,INDIGO.NS,1853.0,-3.98,0.93,-0.44,2125.0,1549.0,52.80,47.20,26.0,2176.0,30.7,1000.00,17.0,15.0,16.0,1.0,-1.0,O
